DATA CLEANING

***

In [1]:
# load packages
import pandas as pd
import numpy as np
import warnings
from collections import Counter


# ignore warnings
warnings.filterwarnings('ignore')

# float format
pd.options.display.float_format = '{:,}'.format

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# load the dataset 
filepath = '/content/drive/MyDrive/Colab Notebooks/export-Viewership20190415.dsv'
data = pd.read_csv(filepath, header=0, sep = "|")
data.shape

(1697179, 15)

In [4]:
# Create a copy of the initial dataframe 
data_clean = data.copy()
data_clean.shape

(1697179, 15)

In [5]:
# Check the data types
data.dtypes

USER_KEY              int64
USER_NAME            object
USER_AGE              int64
USER_SEX             object
VIEW_DATE            object
ASSET_TYPE           object
ASSET_CATEGORY       object
SRC_GENRE_DESC       object
CHANNEL              object
ASSET_NAME           object
PRG_DURATION_MIN     object
VIEW_DURATION_MIN    object
VIEWERSHIP_KEY        int64
ASSET_KEY             int64
VIEW_RATING           int64
dtype: object

In [6]:
# Check data for NAs 
data_clean.isna().sum()

USER_KEY                   0
USER_NAME                313
USER_AGE                   0
USER_SEX                   0
VIEW_DATE                  0
ASSET_TYPE                 0
ASSET_CATEGORY             0
SRC_GENRE_DESC        390728
CHANNEL              1342824
ASSET_NAME             25064
PRG_DURATION_MIN           0
VIEW_DURATION_MIN          0
VIEWERSHIP_KEY             0
ASSET_KEY                  0
VIEW_RATING                0
dtype: int64

In [7]:
# Create subset_USER_NAME - check USER_NAME with 313 NAs
subset_USER_NAME = data_clean[data_clean["USER_NAME"].isna()]
subset_USER_NAME.shape

(313, 15)

In [8]:
# Check subset_USER_NAME -> is the same user for all the 313 rows 
subset_USER_NAME["USER_KEY"].unique()

array([-1])

In [9]:
# Check Nas in subset - there are also NAs in column CHANNEL - we do not want to impute "Uknown" there
subset_USER_NAME.isna().sum()

USER_KEY               0
USER_NAME            313
USER_AGE               0
USER_SEX               0
VIEW_DATE              0
ASSET_TYPE             0
ASSET_CATEGORY         0
SRC_GENRE_DESC         3
CHANNEL              313
ASSET_NAME             0
PRG_DURATION_MIN       0
VIEW_DURATION_MIN      0
VIEWERSHIP_KEY         0
ASSET_KEY              0
VIEW_RATING            0
dtype: int64

In [10]:
##################################################################
#~~~~~~~~~~~~~~~~~~~~~ USER_NAME IMPUTATION~~~~~~~~~~~~~~~~~~~~~ #
##################################################################

In [11]:
# Check in the original dataset - user with id -1 has 313 entries and it has not no USER_NAME too
data_clean[data_clean["USER_KEY"] == -1]
data_clean[data_clean["USER_KEY"] == -1].shape

(313, 15)

In [12]:
# Imputation only in the column USER_NAME
subset_USER_NAME["USER_NAME"] = subset_USER_NAME["USER_NAME"].fillna("UNKNOWN")

In [13]:
# Imputation in data_clean 
data_clean["USER_NAME"] = data_clean["USER_NAME"].fillna("UNKNOWN")

In [14]:
# data_clean - check USER_NAME is ok 
data_clean.isna().sum()

USER_KEY                   0
USER_NAME                  0
USER_AGE                   0
USER_SEX                   0
VIEW_DATE                  0
ASSET_TYPE                 0
ASSET_CATEGORY             0
SRC_GENRE_DESC        390728
CHANNEL              1342824
ASSET_NAME             25064
PRG_DURATION_MIN           0
VIEW_DURATION_MIN          0
VIEWERSHIP_KEY             0
ASSET_KEY                  0
VIEW_RATING                0
dtype: int64

In [15]:
###################################################################
#~~~~~~~~~~~~~~~~~~~~~ ASSET_NAME IMPUTATION~~~~~~~~~~~~~~~~~~~~~ #
###################################################################

In [16]:
# Create subset_SRC_GENRE_DESC - check SRC_GENRE_DESC, CHANNEL, ASSET_NAME  NAs 

subset_SRC_GENRE_DESC = data_clean[data_clean["ASSET_NAME"].isna()]
subset_SRC_GENRE_DESC.shape

(25064, 15)

In [17]:
# Check if  SRC_GENRE_DESC and CHANNEL are missing also the information 
subset_SRC_GENRE_DESC.isna().sum()

USER_KEY                 0
USER_NAME                0
USER_AGE                 0
USER_SEX                 0
VIEW_DATE                0
ASSET_TYPE               0
ASSET_CATEGORY           0
SRC_GENRE_DESC       25064
CHANNEL              25064
ASSET_NAME           25064
PRG_DURATION_MIN         0
VIEW_DURATION_MIN        0
VIEWERSHIP_KEY           0
ASSET_KEY                0
VIEW_RATING              0
dtype: int64

In [18]:
# Set mask_asset_name - filter in order to get the rows with NA values at ASSET_NAME column
mask_asset_name = data_clean['ASSET_NAME'].isna()

# IMPUTATION - update the data_clean dataframe 
data_clean.loc[mask_asset_name, ['ASSET_NAME', 'CHANNEL','SRC_GENRE_DESC']] = 'UNKNOWN'

In [19]:
# data_clean - check ASSET_NAME is ok in 
data_clean.isna().sum()

USER_KEY                   0
USER_NAME                  0
USER_AGE                   0
USER_SEX                   0
VIEW_DATE                  0
ASSET_TYPE                 0
ASSET_CATEGORY             0
SRC_GENRE_DESC        365664
CHANNEL              1317760
ASSET_NAME                 0
PRG_DURATION_MIN           0
VIEW_DURATION_MIN          0
VIEWERSHIP_KEY             0
ASSET_KEY                  0
VIEW_RATING                0
dtype: int64

In [20]:
################################################################
#~~~~~~~~~~~~~~~~~~~~~ CHANNEL IMPUTATION~~~~~~~~~~~~~~~~~~~~~ #
################################################################

In [21]:
# Create subset_CHANNEL for CHANNEL 
subset_CHANNEL = data_clean[data_clean["CHANNEL"].isna()]
subset_CHANNEL.shape

(1317760, 15)

In [22]:
# Set mask_channnel - filter in order to get the rows with NA values at CHANNEL column
mask_channnel = data_clean['CHANNEL'].isna()

# IMPUTATION - update the data_clean dataframe at column CHANNEL
data_clean.loc[mask_channnel, ['CHANNEL']] = 'UNKNOWN'

In [23]:
# data_clean - check CHANNEL is ok 
data_clean.isna().sum()

USER_KEY                  0
USER_NAME                 0
USER_AGE                  0
USER_SEX                  0
VIEW_DATE                 0
ASSET_TYPE                0
ASSET_CATEGORY            0
SRC_GENRE_DESC       365664
CHANNEL                   0
ASSET_NAME                0
PRG_DURATION_MIN          0
VIEW_DURATION_MIN         0
VIEWERSHIP_KEY            0
ASSET_KEY                 0
VIEW_RATING               0
dtype: int64

In [24]:
#######################################################################
#~~~~~~~~~~~~~~~~~~~~~ ASSET_CATEGORY IMPUTATION~~~~~~~~~~~~~~~~~~~~~ #
#######################################################################

In [25]:
# Problem still with ASSET_CATEGORY - even though it has not NAs it has 379427 "Not Available" labeled values 

ASSET_CATEGORY_COUNT = pd.DataFrame(data_clean["ASSET_CATEGORY"].value_counts())
ASSET_CATEGORY_COUNT.columns = ["COUNT"]
ASSET_CATEGORY_COUNT

,COUNT
MOVIES,573501
Not Available,379427
SERIES,222997
"KIDS,MOVIES",194589
KIDS,148807
CHRISTMAS TIME,125257
DOCUMENTARIES,28038
SHOWS,21251
SPORTS,3312


In [26]:
# Create subset_ASSET_CATEGORY for test IMPUTATION

subset_ASSET_CATEGORY = data_clean.loc[data_clean["ASSET_CATEGORY"] == "Not Available"]
subset_ASSET_CATEGORY.shape

(379427, 15)

In [27]:
# Find most common

common_words = pd.DataFrame(Counter(" ".join(subset_ASSET_CATEGORY["ASSET_NAME"]).split()).most_common(200))
common_words.columns = ["WORD/SYMBOL","COUNT"]
common_words.head(50)

,WORD/SYMBOL,COUNT
0,-,204258
1,Euroleague,72354
2,),60686
3,2018/19,39361
4,Championship,33269
5,Football,31878
6,Greek,31616
7,(L)),31600
8,Ep.,29748
9,Sports,29460


In [28]:
# find most common 100 - tail 50
common_words.tail(50)

,WORD/SYMBOL,COUNT
150,at,1929
151,(L)(ASTERAS,1922
152,(L)(CSKA,1913
153,ANATOMY,1886
154,BARCELONA),1840
155,Mystery,1837
156,PANIONIOS,1831
157,TO,1817
158,I,1813
159,4),1784


In [29]:
# Starting value_counts - 379427 rows labeled as  "Not Available"
subset_ASSET_CATEGORY['ASSET_CATEGORY'].value_counts()

Not Available    379427
Name: ASSET_CATEGORY, dtype: int64

In [30]:
# IMPUTATION  

# CATEGORY -  KIDS
keywords_KIDS = ["Animation","Animation/Cartoon","Children","LOONEY","SPONGEBOB","SQUAREPANTS","MICKEY","SCOOBY","Ladybug",
                "KIDS","Kids","TALES","JERRY","TOM","MASHA","BEAN","OZ","DISNEY","PAW","DUCKTALES","BUNNICULA"]

mask_KIDS = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_KIDS)))

data_clean['ASSET_CATEGORY'][mask_KIDS] = "KIDS"

In [31]:
# IMPUTATION  

# CATEGORY -  MOVIES

keywords_MOVIES = ["Comedy","Drama","Action","Crime","SciFi","Romantic","Adventure","comedy","Music",
                "Family","Thriller","Fantasy","Adult","fellowship","Romance","SPARROW","ARTHUR","COLLATERAL","DICTATOR",
                 "movie","SPIDER-MAN","PLATOON","zohan","Drama","PLATOON","CINE","fiction","movie","OCEANS","MAGNIFICENT",
                "2049","Pawn","TRANSYLVANIA"]

mask_MOVIES = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_MOVIES)))

data_clean['ASSET_CATEGORY'][mask_MOVIES] = "MOVIES"

In [32]:
# IMPUTATION 

# CATEGORY -  SPORTS

keywords_SPORTS = ["Euroleague","Championship","Football","2018/19","(L)","Sports","EUROLEAGUE","CHAMPIONS",
"Novasports2","Νovasports","Matchday","PANATHINAIKOS","OLYMPIACOS","CSKA","LAMIA","SOUROTI","ANADOLU","BARCELONA","Champions",
"XANTHIS","TRIPOLIS","AEK","REAL","FENERBAHCE","DARUSSAFAKA","ZALGIRIS","CANARIA","PAOK","PANETOLIKOS","AEK","INTER",
"OFI","AEL","Basketball","MILANO","Serie","LEAGUE","League","FOOTBALL","BASKONIA","ROMA","EFL","LIGUE","SNOOKER","JUVENTUS",
"MACCABI","BAYERN","PANIONIOS","ΚΗΙΜΚΙ","Atromitos","ATROMITOS","FREESPORTS","HAPOEL","ARSENAL","TOTTENHAM","ATALANTA",
"Aris","GERMAIN","Libertadores","PLATE","BOCA","EREDIVISIE","Eredivisie","Fenerbahce","LAZIO","PARMA","SPAL",
"BOLOGNA","TORINO","MANCHESTER","Barcelona","Madrid","PROMITHEAS","CARABAO","BET","Zvezda","FROSINONE","BUDUCNOST","REYER",
"LEICESTER","OOSTENDE","FIFA","Giannina","SASSUOLO","Sampdoria","ChievoVerona"]

mask_SPORTS = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_SPORTS)))
data_clean['ASSET_CATEGORY'][mask_SPORTS] = "SPORTS"


In [33]:
mask_SPORTS.value_counts()

False    1485166
True      212013
Name: ASSET_NAME, dtype: int64

In [34]:
# IMPUTATION  

# CATEGORY -  SHOWS

keywords_SHOWS = ["Entertainment","Discovery","Kardashians","NEWS","Show","LEMMINGS","GREY'S","News","FOODS","THIEVES","DEGENERES",
                  "MURPHY'S","MASTERCHEF","BEYOND","WEATHER","INVESTIGATION","TALENT","SURVIVOR","Geographic"]

mask_SHOWS = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_SHOWS)))
data_clean['ASSET_CATEGORY'][mask_SHOWS] = "SHOWS"

In [35]:
# IMPUTATION 

# CATEGORY -  SERIES

keywords_SERIES = ["Ep.","Episode","LEMMINGS","GREY'S"]

mask_SERIES = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_SERIES)))
data_clean['ASSET_CATEGORY'][mask_SERIES] = "SERIES"

In [36]:
# IMPUTATION 

# CATEGORY -  CHRISTMAS TIME

keywords_CHRISTMAS = ["CHRISTMAS","Novachristmas"]

mask_CHRISTMAS = data_clean.iloc[:, 9].str.contains(r'\b(?:{})\b'.format('|'.join(keywords_CHRISTMAS)))
data_clean['ASSET_CATEGORY'][mask_CHRISTMAS] = "CHRISTMAS TIME"

In [37]:
# data_clean value_counts for ASSET_CATEGORY  after IMPUTATION procedures

data_clean['ASSET_CATEGORY'].value_counts()

MOVIES            624050
SERIES            381234
SPORTS            200892
KIDS,MOVIES       165991
CHRISTMAS TIME    126399
Not Available      80067
KIDS               79736
SHOWS              34495
DOCUMENTARIES       4315
Name: ASSET_CATEGORY, dtype: int64

In [38]:
#######################################################################
#~~~~~~~~~~~~~~~~~~~~~ SRC_GENRE_DESC IMPUTATION~~~~~~~~~~~~~~~~~~~~~ #
#######################################################################

In [39]:
data_clean.isna().sum()

USER_KEY                  0
USER_NAME                 0
USER_AGE                  0
USER_SEX                  0
VIEW_DATE                 0
ASSET_TYPE                0
ASSET_CATEGORY            0
SRC_GENRE_DESC       365664
CHANNEL                   0
ASSET_NAME                0
PRG_DURATION_MIN          0
VIEW_DURATION_MIN         0
VIEWERSHIP_KEY            0
ASSET_KEY                 0
VIEW_RATING               0
dtype: int64

In [40]:
# Create subset_SRC_GENRE_DESC for SRC_GENRE_DESC - 365664 rows labeled as  "NA"          

subset_SRC_GENRE_DESC = data_clean[data_clean["SRC_GENRE_DESC"].isna()]
subset_SRC_GENRE_DESC.shape

(365664, 15)

In [41]:
# Firstly,  we check the ASSET_CATEGORY column 

# 195106 / 365664 are rows related with SPORTS  

# SPORTS is divided into three labels - so will be labeled either DOCUMENTARY, HIGHLIGHTS, N/A

subset_SRC_GENRE_DESC["ASSET_CATEGORY"].value_counts()

SPORTS            187820
MOVIES             59723
Not Available      55001
SHOWS              30111
KIDS               21508
SERIES             10440
CHRISTMAS TIME      1061
Name: ASSET_CATEGORY, dtype: int64

In [42]:
# IMPUTATION in test subset

# CATEGORY - SRC_GENRE - MOVIES - check ASSET_NAME column

data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Comedy', regex = False), 'COMEDY', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Drama', regex = False), 'DRAMA', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Action', regex = False), 'ACTION', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Crime', regex = False), 'CRIME', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('SciFi', regex = False), 'SCIFI', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Romantic', regex = False), 'ROMANCE', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_NAME.str.contains('Adventure', regex = False), 'ADVENTURE', data_clean.SRC_GENRE_DESC)


# CATEGORY - SRC_GENRE - CATEGORY check ASSET_CATEGORY column
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_CATEGORY.str.contains('KIDS', regex = False), 'KIDS', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_CATEGORY.str.contains('SHOWS', regex = False), 'SHOW', data_clean.SRC_GENRE_DESC)
data_clean['SRC_GENRE_DESC'] = np.where(data_clean.ASSET_CATEGORY.str.contains('SPORTS', regex = False), 'SPORTS-LIVE', data_clean.SRC_GENRE_DESC)

In [43]:
data_clean.isna().sum()

USER_KEY                 0
USER_NAME                0
USER_AGE                 0
USER_SEX                 0
VIEW_DATE                0
ASSET_TYPE               0
ASSET_CATEGORY           0
SRC_GENRE_DESC       79200
CHANNEL                  0
ASSET_NAME               0
PRG_DURATION_MIN         0
VIEW_DURATION_MIN        0
VIEWERSHIP_KEY           0
ASSET_KEY                0
VIEW_RATING              0
dtype: int64

In [44]:
# Imputation with label UNKNOWN the remaining 
data_clean['SRC_GENRE_DESC'] = data_clean['SRC_GENRE_DESC'].fillna("UNKNOWN")

In [45]:
data_clean.isna().sum()

USER_KEY             0
USER_NAME            0
USER_AGE             0
USER_SEX             0
VIEW_DATE            0
ASSET_TYPE           0
ASSET_CATEGORY       0
SRC_GENRE_DESC       0
CHANNEL              0
ASSET_NAME           0
PRG_DURATION_MIN     0
VIEW_DURATION_MIN    0
VIEWERSHIP_KEY       0
ASSET_KEY            0
VIEW_RATING          0
dtype: int64

In [46]:
# export csv
# data_clean.to_csv(r'C:\\Users\\xxx\\xxx\\xxx\\data_cleaning.csv')